In [1]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2


In [2]:
os.getcwd()

'/home/aiden/src/dacon_14'

In [3]:
base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

(297571, 3)

### Mecab

In [4]:
mecab = Mecab()
# df_space['morphs'] = df_space['spacing'].apply(lambda x: mecab.morphs(x))
df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))

### tfidf

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_size = 100
stop_words = [
#    '은행',
#     '고객',
#     '가능',
#     '전화',
#     '기간',
#     '대출',
#     '금리',
#     '상담',
#     '광고',
#     '상품',
#     '센터',
]
vectorizer = TfidfVectorizer(max_features=tfidf_size, stop_words=stop_words)

tfidf = vectorizer.fit_transform(df_fea['nouns'].apply(lambda x: ' '.join(x))).toarray()
tfidf_dict = {'tfidf_{0:03d}'.format(v):'tfidf_{}_{}'.format(v, k) for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}
tfidf_cols = sorted(tfidf_dict.keys())

df_tfidf = pd.DataFrame(tfidf, columns=tfidf_cols)

### Merged

In [6]:
df_fea.shape, df_tfidf.shape

((297571, 4), (297571, 100))

In [7]:
df_merged = pd.concat([df_fea.reset_index(), df_tfidf[tfidf_cols]], axis=1)
df_merged.shape

(297571, 105)

In [8]:
df_merged.head(10)

,id,year_month,text,smishing,nouns,tfidf_000,tfidf_001,tfidf_002,tfidf_003,tfidf_004,...,tfidf_090,tfidf_091,tfidf_092,tfidf_093,tfidf_094,tfidf_095,tfidf_096,tfidf_097,tfidf_098,tfidf_099
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,"[행성, 산, 팀장, 행복, 주말]",0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.678718,0.000000,0.000000,0.000000,0.440442,0.0,0.000000,0.000000
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,"[오늘, 하루, 시작, 은행, 진월동, 라운지]",0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.435313,0.000000,0.000000,0.0,0.000000,0.000000
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,"[안녕, 고객, 은행, 금일, 납부, 금액, 원, 감사, 새해, 복, 은행, 옥포]",0.000000,0.0,0.0,0.296568,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,"[고객, 안녕, 은행, 지점, 해, 동안, 저희, 지점, 성원, 감사, 시작, 년,...",0.000000,0.0,0.0,0.142966,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.323314,0.0,0.000000,0.000000
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,"[월, 한, 행복, 해]",0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000
5,6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,"[행복, 주말, 현남, 전담, 직원, 대리]",0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346004,0.0,0.000000,0.000000
6,7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 ...,0,"[고객, 안녕, 은행, 무교, 지점, 과장, 아침, 눈, 눈, 세상, 적, 눈, 눈...",0.000000,0.0,0.0,0.360218,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하...,0,"[고객, 한, 은혜, 진심, 감사, 가슴, 간직, 정유, 년, 새해, 가족, 행복,...",0.000000,0.0,0.0,0.509060,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.575615,0.0,0.000000,0.000000
8,9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,"[설, 연휴, 가족, 정, 추억, 오]",0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준...,1,"[광고, 고객, 뒤, 마음가짐, 준비, 당, 행상, 품, 자격, 기준, 심사, 기준...",0.251901,0.0,0.0,0.000000,0.0,...,0.0,0.126803,0.000000,0.062682,0.000000,0.368754,0.000000,0.0,0.065059,0.130257


In [9]:
cat_cols = []
fea_cols =  tfidf_cols + cat_cols

fea_cols

['tfidf_000',
 'tfidf_001',
 'tfidf_002',
 'tfidf_003',
 'tfidf_004',
 'tfidf_005',
 'tfidf_006',
 'tfidf_007',
 'tfidf_008',
 'tfidf_009',
 'tfidf_010',
 'tfidf_011',
 'tfidf_012',
 'tfidf_013',
 'tfidf_014',
 'tfidf_015',
 'tfidf_016',
 'tfidf_017',
 'tfidf_018',
 'tfidf_019',
 'tfidf_020',
 'tfidf_021',
 'tfidf_022',
 'tfidf_023',
 'tfidf_024',
 'tfidf_025',
 'tfidf_026',
 'tfidf_027',
 'tfidf_028',
 'tfidf_029',
 'tfidf_030',
 'tfidf_031',
 'tfidf_032',
 'tfidf_033',
 'tfidf_034',
 'tfidf_035',
 'tfidf_036',
 'tfidf_037',
 'tfidf_038',
 'tfidf_039',
 'tfidf_040',
 'tfidf_041',
 'tfidf_042',
 'tfidf_043',
 'tfidf_044',
 'tfidf_045',
 'tfidf_046',
 'tfidf_047',
 'tfidf_048',
 'tfidf_049',
 'tfidf_050',
 'tfidf_051',
 'tfidf_052',
 'tfidf_053',
 'tfidf_054',
 'tfidf_055',
 'tfidf_056',
 'tfidf_057',
 'tfidf_058',
 'tfidf_059',
 'tfidf_060',
 'tfidf_061',
 'tfidf_062',
 'tfidf_063',
 'tfidf_064',
 'tfidf_065',
 'tfidf_066',
 'tfidf_067',
 'tfidf_068',
 'tfidf_069',
 'tfidf_070',
 'tfid

In [10]:
train_size = len(df_train)
print(train_size)

295945


### Model

In [11]:
def eval_summary(y_true, y_score, cut_off=0.5):
    y_pred = y_score.copy()
    y_pred[y_pred > cut_off] = 1
    y_pred[y_pred <= cut_off] = 0

    eval_dict = {}
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score, pos_label=1)
    
    eval_dict['auc'] = metrics.auc(fpr, tpr)
    eval_dict['confusion_matrix'] = metrics.confusion_matrix(y_true, y_pred)
    
    pre, rec, _, _ = metrics.precision_recall_fscore_support(y_true, y_pred, pos_label=1)
    eval_dict['precision'] = pre[1]
    eval_dict['recall'] = rec[1]
    
    return eval_dict

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

lgb_ts = datetime.now().strftime('%Y%m%dT%H%M%S')

initscore_filename = ''
params = {
    'boosting':'gbdt',
#     'boosting':'dart',
    'num_leaves': 15, 
    'objective': 'binary',
    'metric':'auc',
    'num_threads': 16,
    'learning_rate': 0.01,
#     'is_unbalance': True,
    'scale_pos_weight':20,
    'bagging_fraction':0.3,
    'bagging_freq':50,
    'feature_fraction':0.4,
    'initscore_filename':initscore_filename,
#     'lambda_l1':200,
#     'lambda_l2':2000,
    'device_type':'gpu',
}
num_round = 3000

[df_test.drop(c, axis=1, inplace=True) for c in df_test.columns if 'smishing_' in c]

df_test = df_merged.loc[train_size:]

In [13]:
# model_index, hidden_index = train_test_split(range(train_size),     
#     test_size=0.1, random_state=1984)

skf_g = StratifiedKFold(n_splits=11, random_state=1984)
cnt_g = 0

for model_index, hidden_index in skf_g.split(range(train_size), df_train['smishing'].values):
    cnt_g = cnt_g + 1
    df_model = df_merged.iloc[model_index]
    df_hidden = df_merged.iloc[hidden_index]
    print('model_set\n', df_model['smishing'].value_counts())
    print('hidden_set\n', df_hidden['smishing'].value_counts())
    
    [df_model.drop(c, axis=1, inplace=True) for c in df_model.columns if 'smishing_' in c]
    [df_hidden.drop(c, axis=1, inplace=True) for c in df_hidden.columns if 'smishing_' in c]


    X = df_model[fea_cols].values
    y = df_model['smishing'].values
#     print(np.unique(y, return_counts=True))

    X_hidden = df_hidden[fea_cols].values
    y_hidden = df_hidden['smishing'].values
#     print(np.unique(y_hidden, return_counts=True))

    skf = StratifiedKFold(n_splits=10, random_state=8405)
#     print(skf)
    cnt = 0
    train_data = lgb.Dataset(X, label=y, feature_name=fea_cols, categorical_feature=cat_cols, free_raw_data=False)

    for train_index, valid_index in skf.split(X, y):
        cnt = cnt + 1
        print('\n', cnt_g, '*' * 20, cnt, '*' * 20)

        train_set = train_data.subset(train_index).construct()
        valid_set = train_data.subset(valid_index).construct()
        print('train_set', np.unique(train_set.get_label(), return_counts=True))
        print('valid_set', np.unique(valid_set.get_label(), return_counts=True))
        bst = lgb.train(params, train_set, num_round, categorical_feature=cat_cols,
                        early_stopping_rounds=300, 
                        valid_sets=[train_set, valid_set],
                        verbose_eval=200,
                        # init_model=init_model,
                       )
        
        renamed_cols = [tfidf_dict[c] if c in tfidf_dict.keys() else c for c in fea_cols]
        impt_dict = {k:v for k, v in zip(renamed_cols, bst.feature_importance(importance_type='split'))}
        print('split:', sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True)[:5])
        
        impt_dict = {k:v for k, v in zip(renamed_cols, bst.feature_importance(importance_type='gain'))}
        print('gain:', sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True)[:5])
 
        
    #     for r in sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True):
    #         print(r)
    #         if r[1] < 1:
    #             del_fea_cols.append(r[0])
    #     print(del_fea_cols)
    #     print(len(del_fea_cols))
    #     lgb.plot_importance(bst, height=0.3, figsize=(20, 100), max_num_features=100)
        df_model['smishing_{}_{}'.format(cnt_g, cnt)] = bst.predict(X)        
        df_hidden['smishing_{}_{}'.format(cnt_g, cnt)] = bst.predict(X_hidden)        
        df_test['smishing_{}_{}'.format(cnt_g, cnt)] = bst.predict(df_test[fea_cols].values)    
        
        print('model\n', eval_summary(y, df_model['smishing_{}_{}'.format(cnt_g, cnt)].values, cut_off=0.5))
        print('hidden\n', eval_summary(y_hidden, df_hidden['smishing_{}_{}'.format(cnt_g, cnt)].values, cut_off=0.5))

    # Predict 정리
    y = df_hidden['smishing'].values
    pred = df_hidden[[c for c in df_hidden.columns if 'smishing_' in c]].mean(axis=1)
    
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    print('\n', '#' * 10, cnt_g, 'auc:', metrics.auc(fpr, tpr))

model_set
 0    252038
1     17002
Name: smishing, dtype: int64
hidden_set
 0    25204
1     1701
Name: smishing, dtype: int64

 1 ******************** 1 ********************
train_set (array([0., 1.], dtype=float32), array([226834,  15301]))
valid_set (array([0., 1.], dtype=float32), array([25204,  1701]))
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999933	valid_1's auc: 0.999909
[400]	training's auc: 0.999943	valid_1's auc: 0.999958
[600]	training's auc: 0.999976	valid_1's auc: 0.999985
[800]	training's auc: 0.999992	valid_1's auc: 0.999993
[1000]	training's auc: 0.999996	valid_1's auc: 0.999996
[1200]	training's auc: 0.999996	valid_1's auc: 0.999997
[1400]	training's auc: 0.999996	valid_1's auc: 0.999998
[1600]	training's auc: 0.999997	valid_1's auc: 0.999997
[1800]	training's auc: 0.999998	valid_1's auc: 0.999982
Early stopping, best iteration is:
[1548]	training's auc: 0.999997	valid_1's auc: 0.999998
split: [('tfidf_63_은행', 887), ('tfidf_

/home/aiden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aiden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aiden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

model
 {'auc': 0.9999967441046982, 'confusion_matrix': array([[252036,      2],
       [    22,  16980]]), 'precision': 0.9998822282416676, 'recall': 0.9987060345841665}
hidden
 {'auc': 0.9999838122799205, 'confusion_matrix': array([[25204,     0],
       [    9,  1692]]), 'precision': 1.0, 'recall': 0.9947089947089947}

 1 ******************** 2 ********************
train_set (array([0., 1.], dtype=float32), array([226834,  15301]))
valid_set (array([0., 1.], dtype=float32), array([25204,  1701]))
Training until validation scores don't improve for 300 rounds
[200]	training's auc: 0.999959	valid_1's auc: 0.999715
Early stopping, best iteration is:
[86]	training's auc: 0.999939	valid_1's auc: 0.999772
split: [('tfidf_11_고객', 50), ('tfidf_79_직원', 44), ('tfidf_43_사항', 37), ('tfidf_44_상담', 36), ('tfidf_45_상품', 35)]
gain: [('tfidf_0_가능', 2228001.746902466), ('tfidf_45_상품', 1677280.5234375), ('tfidf_13_광고', 576725.7120666504), ('tfidf_54_신청', 305436.56645965576), ('tfidf_44_상담', 288541.36199

KeyboardInterrupt: 

In [ ]:
pred_cols = [c for c in df_test.columns if 'smishing_' in c]
print(len(pred_cols))
df_test['pred_max'] = df_test[pred_cols].max(axis=1)
df_test['pred_min'] = df_test[pred_cols].min(axis=1)
df_test['pred_mean'] = df_test[pred_cols].mean(axis=1)
df_test['pred_std'] = df_test[pred_cols].std(axis=1)

In [ ]:
df_test['pred_std'].max()

In [ ]:
df_test['pred_std'].min()

In [ ]:
df_test['pred_mean'].hist(bins=100)

In [ ]:
lgb_ts

In [ ]:
df_test['smishing'] = df_test['pred_mean']
df_test[['id', 'smishing']].to_csv('{}.csv'.format(lgb_ts), index=False)
df_test[['id', 'smishing', 'text']].sort_values('smishing', ascending=False).to_csv('{}_text.csv'.format(lgb_ts), index=False)

In [ ]:
# eval_hist = lgb.cv(params, train_data, num_boost_round=num_round, nfold=20, stratified=True, shuffle=True, 
#        metrics=None, fobj=None, feval=None, init_model=None, feature_name='auto', categorical_feature='auto',
#        early_stopping_rounds=500, fpreproc=None, verbose_eval=100, show_stdv=True, seed=0, callbacks=None,
#        eval_train_metric=False)


In [ ]:
# for train_index, valid_index in skf.split(X, y):
#     cnt = cnt + 1
#     print('\n', '*' * 20, cnt, '*' * 20)
    
#     # init_bst_name = 'model.txt'
#     # init_model = joblib.load(init_bst_name)
#     # bst = None
#     # init_model = bst
#     train_set = train_data.subset(train_index).construct()
#     valid_set = train_data.subset(valid_index).construct()
#     print('train_set', np.unique(train_set.get_label(), return_counts=True))
#     print('valid_set', np.unique(valid_set.get_label(), return_counts=True))
#     bst = lgb.train(param, train_set, num_round, categorical_feature=cat_cols,
#                     early_stopping_rounds=200, 
#                     valid_sets=[train_set, valid_set],
#                     verbose_eval=200,
#                     # init_model=init_model,
#                    )
#     impt_dict = {k:v for k, v in zip(fea_cols, bst.feature_importance())}
#     del_fea_cols = []
#     print(sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True)[:5])
# #     for r in sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True):
# #         print(r)
# #         if r[1] < 1:
# #             del_fea_cols.append(r[0])
# #     print(del_fea_cols)
# #     print(len(del_fea_cols))
# #     lgb.plot_importance(bst, height=0.3, figsize=(20, 100), max_num_features=100)
#     df_model['smishing_{}'.format(cnt)] = bst.predict(X)        
#     df_hidden['smishing_{}'.format(cnt)] = bst.predict(X_hidden)        
#     df_test['smishing_{}'.format(cnt)] = bst.predict(df_test[fea_cols].values)    